# Book Recommendation Using Surprise
BAX452 HW7

Data source: http://www2.informatik.uni-freiburg.de/~cziegler/BX/

Other data sets available: https://gist.github.com/entaroadun/1653794

In [4]:
import pandas as pd
import numpy as np
import math
import re
from scipy.sparse import csr_matrix
import matplotlib.pyplot as plt
import seaborn as sns
from surprise import Reader, Dataset, SVD, evaluate, model_selection
sns.set_style("darkgrid")

In [5]:
ratings = pd.read_csv("/Users/liqiongbai/Downloads/Data/BX-CSV-Dump/BX-Book-Ratings.csv", 
                      sep=';',error_bad_lines=False,encoding='latin-1')
ratings.head()

,User-ID,ISBN,Book-Rating
0,276725,034545104X,0
1,276726,0155061224,5
2,276727,0446520802,0
3,276729,052165615X,3
4,276729,0521795028,6


In [6]:
books = pd.read_csv("/Users/liqiongbai/Downloads/Data/BX-CSV-Dump/BX-Books.csv", 
                    sep=';',error_bad_lines=False,encoding='latin-1')
books.head()

b'Skipping line 6452: expected 8 fields, saw 9\nSkipping line 43667: expected 8 fields, saw 10\nSkipping line 51751: expected 8 fields, saw 9\n'
b'Skipping line 92038: expected 8 fields, saw 9\nSkipping line 104319: expected 8 fields, saw 9\nSkipping line 121768: expected 8 fields, saw 9\n'
b'Skipping line 144058: expected 8 fields, saw 9\nSkipping line 150789: expected 8 fields, saw 9\nSkipping line 157128: expected 8 fields, saw 9\nSkipping line 180189: expected 8 fields, saw 9\nSkipping line 185738: expected 8 fields, saw 9\n'
b'Skipping line 209388: expected 8 fields, saw 9\nSkipping line 220626: expected 8 fields, saw 9\nSkipping line 227933: expected 8 fields, saw 11\nSkipping line 228957: expected 8 fields, saw 10\nSkipping line 245933: expected 8 fields, saw 9\nSkipping line 251296: expected 8 fields, saw 9\nSkipping line 259941: expected 8 fields, saw 9\nSkipping line 261529: expected 8 fields, saw 9\n'
/Users/liqiongbai/anaconda/lib/python3.6/site-packages/IPython/core/intera

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...


In [7]:
users = pd.read_csv("/Users/liqiongbai/Downloads/Data/BX-CSV-Dump/BX-Users.csv", 
                    sep=';',error_bad_lines=False,encoding='latin-1')
users.head()

,User-ID,Location,Age
0,1,"nyc, new york, usa",NaN
1,2,"stockton, california, usa",18.0
2,3,"moscow, yukon territory, russia",NaN
3,4,"porto, v.n.gaia, portugal",17.0
4,5,"farnborough, hants, united kingdom",NaN


In [8]:
## combine data sets

# df = ratings.append(users) 
# df = df.append(books)
df = pd.merge(ratings, users, left_on='User-ID', right_on='User-ID', how='left')
df = pd.merge(df, books, left_on='ISBN', right_on='ISBN', how='left')
df.head()

,User-ID,ISBN,Book-Rating,Location,Age,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,276725,034545104X,0,"tyler, texas, usa",NaN,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books,http://images.amazon.com/images/P/034545104X.0...,http://images.amazon.com/images/P/034545104X.0...,http://images.amazon.com/images/P/034545104X.0...
1,276726,0155061224,5,"seattle, washington, usa",NaN,Rites of Passage,Judith Rae,2001,Heinle,http://images.amazon.com/images/P/0155061224.0...,http://images.amazon.com/images/P/0155061224.0...,http://images.amazon.com/images/P/0155061224.0...
2,276727,0446520802,0,"h, new south wales, australia",16.0,The Notebook,Nicholas Sparks,1996,Warner Books,http://images.amazon.com/images/P/0446520802.0...,http://images.amazon.com/images/P/0446520802.0...,http://images.amazon.com/images/P/0446520802.0...
3,276729,052165615X,3,"rijeka, n/a, croatia",16.0,Help!: Level 1,Philip Prowse,1999,Cambridge University Press,http://images.amazon.com/images/P/052165615X.0...,http://images.amazon.com/images/P/052165615X.0...,http://images.amazon.com/images/P/052165615X.0...
4,276729,0521795028,6,"rijeka, n/a, croatia",16.0,The Amsterdam Connection : Level 4 (Cambridge ...,Sue Leather,2001,Cambridge University Press,http://images.amazon.com/images/P/0521795028.0...,http://images.amazon.com/images/P/0521795028.0...,http://images.amazon.com/images/P/0521795028.0...


In [9]:
df.shape

(1149780, 12)

In [10]:
df.isnull().sum()

User-ID                     0
ISBN                        0
Book-Rating                 0
Location                    0
Age                    309492
Book-Title             118644
Book-Author            118644
Year-Of-Publication    118644
Publisher              118646
Image-URL-S            118644
Image-URL-M            118644
Image-URL-L            118648
dtype: int64

In [11]:
## Fill missing age with median
df['Age'].fillna((df['Age'].median()), inplace=True)

In [12]:
## Drop books with missing titles, authors, years of publications, or publishers
df = df.dropna()

In [13]:
## Drop columns of the image URLs
df = df.drop(['Image-URL-S','Image-URL-M','Image-URL-L'], axis=1)

In [14]:
df.shape

(1031130, 9)

In [15]:
df = df[df['Book-Rating'] != 0]; df.shape

(383839, 9)

In [17]:
## Recommend with Collaborative Filtering
reader = Reader()

# get just top 100K rows for faster run time
data = Dataset.load_from_df(df[['User-ID', 'ISBN', 'Book-Rating']][:100000], reader)
data.split(n_folds=3)

svd = SVD()
evaluate(svd, data, measures=['RMSE', 'MAE'])

/Users/liqiongbai/anaconda/lib/python3.6/site-packages/surprise/evaluate.py:66: UserWarning: The evaluate() method is deprecated. Please use model_selection.cross_validate() instead.
  'model_selection.cross_validate() instead.', UserWarning)
/Users/liqiongbai/anaconda/lib/python3.6/site-packages/surprise/dataset.py:193: UserWarning: Using data.split() or using load_from_folds() without using a CV iterator is now deprecated. 
  UserWarning)


Evaluating RMSE, MAE of algorithm SVD.

------------
Fold 1
RMSE: 3.2270
MAE:  2.8108
------------
Fold 2
RMSE: 3.2108
MAE:  2.7904
------------
Fold 3
RMSE: 3.2328
MAE:  2.8224
------------
------------
Mean RMSE: 3.2235
Mean MAE : 2.8079
------------
------------


CaseInsensitiveDefaultDict(list,
                           {'mae': [2.8108131294966312,
                             2.790363302044732,
                             2.8223762938899783],
                            'rmse': [3.2270229547391693,
                             3.2107764493893809,
                             3.2328168592591151]})

In [34]:
## Below is what user 276729 liked (rating>=5) in the past:
df_276729 = df[(df['User-ID'] == 276729) & (df['Book-Rating'] >= 5)]
df_276729 = df_276729.set_index('ISBN')
# df_276729 = df_276729.join(books)['Book-Title']
df_276729

,User-ID,Book-Rating,Location,Age,Book-Title,Book-Author,Year-Of-Publication,Publisher
ISBN,,,,,,,,
0521795028,276729,6,"rijeka, n/a, croatia",16.0,The Amsterdam Connection : Level 4 (Cambridge ...,Sue Leather,2001,Cambridge University Press


In [37]:
## predict which book user 276729 would love to give a rating above 5:
user_276729 = df.copy()
user_276729 = user_276729.reset_index()

In [38]:
# getting full dataset
dataset = Dataset.load_from_df(df[['User-ID', 'ISBN', 'Book-Rating']], reader)

trainset = dataset.build_full_trainset()
svd.train(trainset)

user_276729['Estimate_Score'] = user_276729['ISBN'].apply(lambda x: svd.predict(276729, x).est)

user_276729 = user_276729.drop('ISBN', axis = 1)

user_276729 = user_276729.sort_values('Estimate_Score', ascending=False)
user_276729.head()

/Users/liqiongbai/anaconda/lib/python3.6/site-packages/surprise/prediction_algorithms/algo_base.py:51: UserWarning: train() is deprecated. Use fit() instead
  warnings.warn('train() is deprecated. Use fit() instead', UserWarning)


,index,User-ID,Book-Rating,Location,Age,Book-Title,Book-Author,Year-Of-Publication,Publisher,Estimate_Score
0,1,276726,5,"seattle, washington, usa",35.0,Rites of Passage,Judith Rae,2001,Heinle,5.0
255759,744001,179931,9,"toronto, ontario, canada",37.0,Moon Palace (Contemporary American Fiction),Paul Auster,1990,Penguin Books,5.0
255768,744029,179939,9,"barcelona, granollers, spain",27.0,El Palacio de La Luna,Paul Auster,1996,Anagrama,5.0
255767,744027,179939,7,"barcelona, granollers, spain",27.0,Sin Noticias De Gurb (Biblioteca breve),Eduardo Mendoza,1995,Planeta Pub Corp,5.0
255766,744026,179939,9,"barcelona, granollers, spain",27.0,El Guardian Entre El Centeno,J. D. Salinger,1999,AIMS International Books,5.0
